In [155]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

import inspect, os
currentdir = os.path.dirname(os.path.abspath(inspect.getfile(inspect.currentframe())))
parentdir = os.path.dirname(currentdir)
os.sys.path.insert(0,parentdir) 

from src.survey_utils import *
import copy
import pandas as pd
import numpy as np

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [156]:
d = pd.read_csv('../data/reader-survey-3.tsv', sep = '\t')

In [167]:
d = d.fillna('no_response')

In [168]:
d_q = {'I am reading this article to': 'Information_Need',
       'Prior to visiting this article':'Prior_Knowledge',
    'I am reading this article because' :  'Motivation'
             }


d_a1 = {'look up a specific fact or to get a quick answer.': 'fact',
       'get an overview of the topic.': 'overview',
       'get an in-depth understanding of the topic.': 'in-depth',
       'no_response': 'no_response'}

d_a2 = {'I was already familiar with the topic.': 'familiar',
       'I was not familiar with the topic and I am learning about it for the first time.': 'unfamiliar',
        'no_response': 'no_response'}
d_a3 = {'I have a work or school-related assignment.' : 'work/school',
       'I need to make a personal decision based on this topic (e.g. to buy a  book or game, to choose a travel destination).': 'personal_decision',
       "I want to know more about\xa0a current event (e.g. Black Friday, a soccer game, a recent earthquake, somebody's death).": 'current_event',
       "the topic was referenced in a piece of media (e.g. TV, radio, article, film, book).": 'media',
       "the topic came up in a conversation.": "conversation",
       "I am bored, curious or randomly exploring Wikipedia for fun.": 'bored/curios/random',
        'no_response': 'no_response'
       }
d_a = {'Information_Need': d_a1, 'Prior_Knowledge': d_a2,'Motivation': d_a3 }

In [179]:
def recode_motivation(x):
        x = str(x)
        x = x.replace('e.g.,', 'e.g.')
        ms = x.split('., ')
        ms = [m + '.' for m in ms]
        ms[-1] = ms[-1][:-1]    
        #print(ms)
        rms = [d_a['Motivation'].get(m, 'other') for m in ms]
        return '|'.join(rms) 
    
def recode_df(d):
    d = copy.deepcopy(d)
    del d['Timestamp']
    d.columns = [d_q[c] for c in d.columns]
    
    d['Information_Need'] = d['Information_Need'].apply(lambda x: d_a['Information_Need'].get(x, 'other'))
    d['Prior_Knowledge'] = d['Prior_Knowledge'].apply(lambda x: d_a['Prior_Knowledge'].get(x, 'other'))
    
    
    d['Motivation'] = d['Motivation'].apply(recode_motivation)

    return d

In [180]:
dr = recode_df(d)

In [187]:
#dr[dr['Motivation2'] == 'bored/curios/random|other']

In [188]:
#dr.ix[14]['Motivation']

In [189]:
recode_motivation(dr.ix[92]['Motivation'])

'other'

In [190]:
dr.to_csv('../app/data/reader-survey-3-recoded.tsv', sep = '\t', index = False)

In [191]:
dr['Motivation'].value_counts()

bored/curios/random                                                                         2073
media                                                                                       1928
work/school                                                                                 1141
conversation                                                                                1006
other                                                                                        598
current_event                                                                                590
personal_decision                                                                            469
media|bored/curios/random                                                                    411
no_response                                                                                  245
conversation|bored/curios/random                                                             149
current_event|bored/curios/ran